----TODO-----
Actualizar feronomonas, sobre todo fitness por pieza
Pruebas Fitness

In [1]:
import copy
#import numpy as np
from typing import List, Dict

In [2]:
class PiezaClase:
    def __init__(self, pieza, x_pos, y_pos):
        self.pieza = pieza
        self.x_pos = x_pos
        self.y_pos = y_pos
    
"""     def _initialize(pieza, x_pos, y_pos):
        self.pieza = pieza
        self.x_pos = x_pos
        self.y_pos = y_pos """

'     def _initialize(pieza, x_pos, y_pos):\n        self.pieza = pieza\n        self.x_pos = x_pos\n        self.y_pos = y_pos '

In [3]:
def explore_adyacent(temp, x, y): #hay que marcar la celda inicial antes de llamarla
    area = 0
    # Verificar si la celda adyacente está dentro de los límites del espacio
    if x + 1 < len(temp) and temp[x+1][y] == 0:
        temp[x+1][y] = 1
        area += explore_adyacent(temp, x+1, y)
    if x - 1 >= 0 and temp[x-1][y] == 0:
        temp[x-1][y] = 1
        area += explore_adyacent(temp, x-1, y)
    if y + 1 < len(temp[0]) and temp[x][y+1] == 0:
        temp[x][y+1] = 1
        area += explore_adyacent(temp, x, y+1)
    if y - 1 >= 0 and temp[x][y-1] == 0:
        temp[x][y-1] = 1
        area += explore_adyacent(temp, x, y-1)
    if temp[x][y]:
        area += 1
    return area


def huecos (mat):
  matrix = copy.deepcopy(mat)
  array = []
  for i in range(len(mat)):
    for j in range(len(mat[0])):
      if matrix[i][j] == 0:
        matrix[i][j] = 1
        array.append(explore_adyacent(matrix, i, j))
  return array

space =  [[0, 1, 0, 1],
          [0, 1, 0, 1],
          [0, 0, 1, 0],
          [0, 1, 1, 1]]
temp_area = copy.deepcopy(space)
array = huecos(temp_area)
print(space)
print (temp_area)
print (array)

[[0, 1, 0, 1], [0, 1, 0, 1], [0, 0, 1, 0], [0, 1, 1, 1]]
[[0, 1, 0, 1], [0, 1, 0, 1], [0, 0, 1, 0], [0, 1, 1, 1]]
[5, 2, 1]


In [4]:
def clamp_to_zero(number):
    if number < 0:
        return 0
    else:
        return number

FITNESS DEFINITION

In [5]:
def fitness(space, pieza, x, y):
  valid = colocar_pieza(space, pieza, x, y)
  espacios = huecos(space)
  if not valid:
    return 0
  else:
    value = 100
    value = value - len(espacios) * 5 #penalizar numero de huecos
    for hueco in espacios:
      value = value - (1/hueco) * 3
    return clamp_to_zero(value)

In [6]:
def fitnessSinPieza(space):
    espacios = huecos(space)
    value = 100
    value = value - len(espacios) * 5 #penalizar numero de huecos
    for hueco in espacios:
      value = value - (1/hueco) * 3
    return clamp_to_zero(value)

In [7]:
def colocar_pieza(space, pieza, x, y):
    temp = copy.deepcopy(space)  # Deep copy the space to avoid modifying the original
    if len(pieza)+x >= len(space) or len(pieza[0])+y >= len(space[0]):
                return False  # Return False if the piece goes out of bounds
    for i in range(len(pieza)):
        for j in range(len(pieza[0])):
            # Check bounds
            if space[x+i][y+j] == 1:
                return False  # Return False if there's an overlap
            temp[x+i][y+j] = 1  # Place the piece in the temporary space
    # Copy the modified temporary space back to the original space
    for i in range(len(space)):
        for j in range(len(space[0])):
            space[i][j] = temp[i][j]
    return True  # Return True if the piece was successfully placed

In [8]:
#Función para crear la pieza, empezamos creando piezas de un máximo de area de 3x3, para empezar con una versión sencilla del problema, haremos que las piezas inicialmente solo puedan ser cuadrados de 1x1, 2x2 o 3x3.

import random

def generar_pieza():
    # Inicializamos una matriz vacía
    sizex = random.randint(1, 6)
    sizey = random.randint(1, 6)
    pieza = [[1 for column in range(sizex)] for row in range(sizey)]
    return pieza


def imprimir_pieza(pieza:PiezaClase):
    for row in pieza.pieza:
        print("  ".join(map(str, row)))
    print("coordenadas: x:",pieza.x_pos, "y:",pieza.y_pos)
    print()

def imprimir_espacio(space):
    for row in space:
        print("  ".join(map(str, row)))
    print()



# Ejemplo de uso
if __name__ == "__main__":
    pieza = PiezaClase(generar_pieza(),random.randint(1, 6),random.randint(1, 6))
    imprimir_pieza(pieza)

1
1
coordenadas: x: 3 y: 4



In [9]:

x = 16
y = 16
space = [[0 for column in range(x)] for row in range(y)]

x_pos = 0
y_pos = 0
#print(x_pos,y_pos)
#pieza = generar_pieza()
pieza2 = PiezaClase(generar_pieza(),0,0)
imprimir_pieza(pieza2)
print(fitness(space, pieza2.pieza, pieza2.x_pos, pieza2.y_pos))


print("inicial: ")
imprimir_espacio(space)
temp_area = copy.deepcopy(space)
array = huecos(temp_area)

print (temp_area)
print (array)

1  1  1  1  1
1  1  1  1  1
coordenadas: x: 0 y: 0

94.98780487804878
inicial: 
1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0
1  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

[[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 

PRIMERA IMPLEMENTACION DE ACO

In [10]:
#Genero el espacio donde colocamos las piezas
x = 16
y = 16
space = [[0 for column in range(x)] for row in range(y)]

#Genero un array de piezas
piezas = []
for i in range(3):
    piezaIndividual = PiezaClase(generar_pieza(),random.randint(0, 15),random.randint(0, 15))
    piezas.append(piezaIndividual)
    
    #imprimir_pieza(piezas[i])

In [11]:
""" def generar_sol(space):
    temp = PiezaClase
    sol = []
    for i in range(10):
        temp = piezas[random.randint(0,9)] #hacerlo clase PiezaClase
        if (fitness(space, temp.pieza, temp.x_pos, temp.y_pos) != 0):
            sol.append(temp)
    return(sol) """

' def generar_sol(space):\n    temp = PiezaClase\n    sol = []\n    for i in range(10):\n        temp = piezas[random.randint(0,9)] #hacerlo clase PiezaClase\n        if (fitness(space, temp.pieza, temp.x_pos, temp.y_pos) != 0):\n            sol.append(temp)\n    return(sol) '

In [12]:
def heuristico(piezaHeu:PiezaClase): #se puede hacer un heuristico que varíe segun avanza.
    size = len(piezaHeu.pieza)*len(piezaHeu.pieza[0])
    
    return size / (x*x)

In [13]:
def calculoProbabilidades(pheromonesCopy, piezas):
    alpha = 0.5
    beta = 0.5
    dividendo = []
    suma = 0
    for i in range(0, len(piezas)):
        dividendo.append(pow(pheromonesCopy[i],alpha) * pow(heuristico(piezas[i]),beta))
        suma += dividendo[i]
    for j in dividendo:
        j = j/suma
    #print("Dividendo",dividendo)
    return dividendo

In [14]:
#El array sol contiene n arrays, donde n es len de piezas, en los cuales el primer valor es
# si se pone o no se pone le pieza y los otros dos la posicion
# En caso de que se ponga se intenta colocar por ahora de manera aleatoria 50 veces, si no se consigue se pone como False
# esto despues se hara con feromonas y tal pero por ahora random
def generar_sol(space, pheromones):
    sol = [[0 for column in range(0,3)] for row in range(len(piezas))]
    probabilidades=calculoProbabilidades(copy.deepcopy(pheromones), piezas)
    for i in range(len(piezas)):
        if(random.random() < probabilidades[i]):
            sol[i][0] = True
        else:
            sol[i][0] = False
        if (sol[i][0] == True):
            for j in range (0,50):
                posTempX = random.randint(0, 15)
                posTempY = random.randint(0, 15)
                if(fitness(space,piezas[i].pieza,posTempX, posTempY) != 0):
                    sol[i][1] = posTempX
                    sol[i][2] = posTempY
                    break
                elif(j == 10):
                    sol[i][0] = False
                    sol[i][1] = -1
                    sol[i][2] = -1
        else:
            sol[i][1] = -1
            sol[i][2] = -1
    return sol

# print(generar_sol(space))
# imprimir_espacio(space)
# print(fitnessSinPieza(space))

In [30]:
def fitnesFeromona(caminos):
    sol = [0 for i in range(len(pheromones))]
    for i in range(len(caminos)):
        for j in range(len(piezas)):
            if(caminos[i][j][0] == True):
                sol[j] = sol[j] + 1
    for x in sol:
        x = x/len(caminos)
    return sol

In [36]:
def actualizarFeromonas(pheromones, caminos): #Esta mal, no se como hacer bien lo de los caminos
    evap = 0.5
    fitFeromona = fitnesFeromona(caminos)
    for i in range(0,len(pheromones)):
        if pheromones[i] > 0.5:
            pheromones[i] = (1-evap)*pheromones[i] + fitFeromona[i]

In [39]:
#Genero el ACO
pheromones = [random.random()+0.5 for i in range(0,len(piezas))]
best_solution = []
numberAnts = 10
for i in range(0,15):
    caminos = []
    for j in range(numberAnts):
        solution = generar_sol(copy.deepcopy(space), pheromones)
        #print(fitnessSinPieza(solution), solution) #TO DO, imprimir space con esta sol para leerlo mejor
        caminos.append(solution)
        #print(caminos)
        fit = fitnessSinPieza(solution)
        if(fit > fitnessSinPieza(best_solution)):
            best_solution = solution
    actualizarFeromonas(pheromones,caminos)
    print("pheromones", pheromones)

#imprimir_espacio(space)
print(best_solution)

pheromones [1.4565172227125356, 0.5631642262245514, 4.535681331537979]
pheromones [1.7282586113562677, 2.2815821131122758, 8.26784066576899]
pheromones [1.8641293056781338, 1.1407910565561379, 13.133920332884495]
pheromones [1.9320646528390668, 1.5703955282780688, 16.566960166442247]
pheromones [3.9660323264195334, 0.7851977641390344, 18.28348008322112]
pheromones [3.9830161632097667, 1.3925988820695172, 19.14174004161056]
pheromones [1.9915080816048834, 2.6962994410347587, 19.57087002080528]
pheromones [0.9957540408024417, 2.3481497205173794, 19.78543501040264]
pheromones [0.49787702040122084, 2.1740748602586897, 19.89271750520132]
pheromones [0.49787702040122084, 2.087037430129345, 19.94635875260066]
pheromones [0.49787702040122084, 3.0435187150646725, 19.97317937630033]
pheromones [0.49787702040122084, 3.5217593575323365, 19.986589688150165]
pheromones [0.49787702040122084, 3.7608796787661682, 19.99329484407508]
pheromones [0.49787702040122084, 3.880439839383084, 19.99664742203754]
